In [1]:
import pandas as pd
import re
import torch
import numpy as np
import torch
import transformers
from ast import literal_eval
import json
import pandas as pd
import random


pd.set_option('display.max_columns', None)

In [2]:
!pip install openai==0.28

In [3]:
df_csv = pd.read_csv("last_data.csv")

In [ ]:
import openai
openai.api_key = ""

In [5]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42) 

In [6]:
df = df_csv.copy()

In [7]:
# 사회 영역과 국어 영역 데이터 분리
social_sample = df[df['cleaned_category'].str.startswith('사회')]
korean_sample = df[df['cleaned_category'].str.startswith('국어')]

In [8]:
# set_seed(42) 
# # 각 영역에서 2개씩 샘플 추출
# social_sample = social_df.sample(n=900)
# korean_sample = korean_df.sample(n=100)
# social_sample

In [9]:
def generate_question(row, subject):
    prompt = f"""당신은 한국의 {subject} 교육과정과 공무원 시험에 맞는 문제를 만드는 전문가입니다. 다음 지침을 엄격히 따라 새로운 문제를 만들어주세요:

1. 주어진 문제와 유사한 난이도와 형식을 유지하되, 내용은 다르게 만드세요.
2. {subject}의 특성을 반영하여 문제를 구성하세요.
3. 고등학교 교육과정부터 7급 공무원 시험 수준까지의 범위에서 출제하세요.
4. 문제는 반드시 다음 요소를 모두 포함해야 합니다. 어느 하나라도 비어있으면 안 됩니다:
   - 지문: 최소 100자 이상의 의미 있는 텍스트
   - 질문: 지문을 바탕으로 한 명확한 질문
   - 선택지: 정확히 5개의 선택지 (각 선택지는 최소 5단어 이상)
   - 정답: 1에서 5 사이의 숫자
   - 카테고리: 구체적인 과목 영역
5. 정답은 반드시 1에서 5 사이의 숫자여야 합니다.
6. 정답의 분포가 균등하도록 노력해주세요. 즉, 1부터 5까지의 답이 비슷한 빈도로 나오도록 해주세요.

참고할 기존 문제:
지문: {row['paragraph']}
질문: {row['question']}
선택지: {row['choices']}
정답: {row['answer']}
카테고리: {row['cleaned_category']}

{subject}가 사회 과목일 경우:
- 역사 지식의 이해, 연대기 파악, 역사 상황 및 쟁점 인식, 역사 자료 분석 및 해석, 역사 탐구 설계 및 수행, 결론 도출 및 평가 능력 중 하나 이상을 평가하는 문제를 만드세요.
- 경제, 정치와 법, 문화, 윤리와 사상, 세계사, 생활과 윤리, 동아시아사, 한국지리, 교육, 한국사 중 하나의 세부 영역을 선택하여 문제를 구성하세요.
- 한국사의 경우, 고대부터 현대까지의 주요 사건, 인물, 제도 등을 포함할 수 있으며, 7급 공무원 시험 수준의 심화된 내용도 다룰 수 있습니다.

{subject}가 국어 과목일 경우:
- 독서, 문학, 화법, 작문, 언어, 매체 영역 중 하나를 선택하여 문제를 구성하세요.
- 어휘･개념, 사실적 이해, 추론적 이해, 비판적 이해, 적용･창의 능력 중 하나 이상을 평가하는 문제를 만드세요.

위 지침을 바탕으로 새로운 문제를 다음 형식으로 작성해주세요. 모든 필드를 반드시 채워야 합니다:
지문: (새로운 지문 - 최소 100자)
질문: (새로운 질문)
선택지: (5개의 새로운 선택지, 각각 최소 5단어)
정답: (1에서 5 사이의 새로운 정답)
카테고리: (새로운 카테고리)

주의: 모든 필드는 반드시 채워져야 하며, 비어있는 필드가 있어서는 안 됩니다. 형식을 정확히 따라주세요.
"""

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "당신은 한국의 교육과정과 공무원 시험에 맞는 문제를 만드는 전문가입니다. 모든 필드를 빠짐없이 채워야 합니다."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content']

# 새로운 문제 생성
augmented_data = []

# 진행 상황을 표시하는 tqdm 사용
from tqdm import tqdm

# 사회 문제 생성 (450개)
for _, row in tqdm(social_sample.iterrows(), total=len(social_sample), desc="사회 문제 생성"):
    new_question = generate_question(row, "사회")
    augmented_data.append(new_question)

# 국어 문제 생성 (50개)
for _, row in tqdm(korean_sample.iterrows(), total=len(korean_sample), desc="국어 문제 생성"):
    new_question = generate_question(row, "국어")
    augmented_data.append(new_question)

# 생성된 문제 수 확인
print(f"총 생성된 문제 수: {len(augmented_data)}")

# 생성된 문제 출력 (옵션)
for i, question in enumerate(augmented_data, 1):
    print(f"생성된 문제 {i}:")
    print(question)
    print("-" * 50)




국어 문제 생성: 100%|██████████| 113/113 [06:57<00:00,  3.69s/it]

총 생성된 문제 수: 1962
생성된 문제 1:
지문: 조선 후기에 들어서면서 사대부 계층의 정치적 영향력이 점차 늘어나게 되었다. 이들은 국가의 정책 결정에 있어 중요한 역할을 하였으며, 그들 사이에는 다양한 의견과 갈등이 존재했다. 특히, 남인과 북인의 갈등은 정치뿐만 아니라 사회와 경제에도 큰 영향을 미쳤으며, 이는 조선 사회의 변화를 초래했다. 그 과정에서 특정 인물들이 두 진영의 중재 역할을 하면서도 갈등을 더욱 심화시키는 결과를 초래하기도 하였다. 이러한 상황은 결국 명나라와 청나라의 외세 개입으로 이어지는 배경이 되었다.

질문: 조선 후기에 남인과 북인의 갈등이 심화된 원인으로 가장 적절한 것은 무엇인가?
선택지: ['사대부의 정치적 영향력이 커졌다', '경제적 불만이 사회 불안을 초래했다', '외세에 대한 대응 방안이 달랐다', '정치적 중재자 인물의 등장 때문이다', '문화적 차이가 양 진영을 갈랐다']
정답: 3
카테고리: 사회-한국사
--------------------------------------------------
생성된 문제 2:
지문: 고려시대에는 불교가 국가의 주된 사상으로 자리 잡으면서 다양한 불교 사찰과 신앙이 발전하였다. 특히, 왕실과 귀족계층의 후원을 받아 전래된 대승불교는 사회 전반에 걸쳐 큰 영향을 미쳤으며, 이를 통해 국가의 통합과 민중의 안녕이 도모되었다. 그러나 불교가 지배적인 지위를 차지하는 동안, 유교와 도교 등 다른 사상들이 밀접하게 연결되기도 하였다. 이러한 복합적인 신앙 형태는 고려 사회의 특징을 잘 보여준다.

질문: 고려시대의 불교 사상이 미친 영향에 대한 설명으로 옳은 것은 무엇인가?
선택지: ['불교의 영향으로 왕실과 국민의 통합이 이루어졌다.', '대중이 불교를 따르지 않는 혁명이 발생하였다.', '불교가 유교보다 더 우선시된 사례가 많았다.', '불교 사찰의 건축 기술이 급격히 발전했다.', '고려시대 동안 불교가 완전히 사라졌다.']
정답: 1
카테고리: 사회-한국사
-----------

In [10]:
augmented_data

["지문: 조선 후기에 들어서면서 사대부 계층의 정치적 영향력이 점차 늘어나게 되었다. 이들은 국가의 정책 결정에 있어 중요한 역할을 하였으며, 그들 사이에는 다양한 의견과 갈등이 존재했다. 특히, 남인과 북인의 갈등은 정치뿐만 아니라 사회와 경제에도 큰 영향을 미쳤으며, 이는 조선 사회의 변화를 초래했다. 그 과정에서 특정 인물들이 두 진영의 중재 역할을 하면서도 갈등을 더욱 심화시키는 결과를 초래하기도 하였다. 이러한 상황은 결국 명나라와 청나라의 외세 개입으로 이어지는 배경이 되었다.\n\n질문: 조선 후기에 남인과 북인의 갈등이 심화된 원인으로 가장 적절한 것은 무엇인가?\n선택지: ['사대부의 정치적 영향력이 커졌다', '경제적 불만이 사회 불안을 초래했다', '외세에 대한 대응 방안이 달랐다', '정치적 중재자 인물의 등장 때문이다', '문화적 차이가 양 진영을 갈랐다']\n정답: 3\n카테고리: 사회-한국사",
 "지문: 고려시대에는 불교가 국가의 주된 사상으로 자리 잡으면서 다양한 불교 사찰과 신앙이 발전하였다. 특히, 왕실과 귀족계층의 후원을 받아 전래된 대승불교는 사회 전반에 걸쳐 큰 영향을 미쳤으며, 이를 통해 국가의 통합과 민중의 안녕이 도모되었다. 그러나 불교가 지배적인 지위를 차지하는 동안, 유교와 도교 등 다른 사상들이 밀접하게 연결되기도 하였다. 이러한 복합적인 신앙 형태는 고려 사회의 특징을 잘 보여준다.\n\n질문: 고려시대의 불교 사상이 미친 영향에 대한 설명으로 옳은 것은 무엇인가?\n선택지: ['불교의 영향으로 왕실과 국민의 통합이 이루어졌다.', '대중이 불교를 따르지 않는 혁명이 발생하였다.', '불교가 유교보다 더 우선시된 사례가 많았다.', '불교 사찰의 건축 기술이 급격히 발전했다.', '고려시대 동안 불교가 완전히 사라졌다.']\n정답: 1\n카테고리: 사회-한국사",
 '지문: 조선 시대, 성리학이 국가의 이념으로 자리 잡으면서 교육제도의 뼈대를 형성하게 되었다. 그 중에서도 유학이 강조되었고, 사서오경을 중심

In [11]:
augmented_df = pd.DataFrame(columns=['id', 'paragraph', 'question', 'choices', 'answer', 'category', 'question_plus'])

for i, question in enumerate(augmented_data):
    try:
        parts = question.split('\n')
        paragraph = next((part.replace("지문: ", "") for part in parts if part.startswith("지문:")), "")
        question_text = next((part.replace("질문: ", "") for part in parts if part.startswith("질문:")), "")
        choices = next((part.replace("선택지: ", "") for part in parts if part.startswith("선택지:")), "")
        answer = next((part.replace("정답: ", "") for part in parts if part.startswith("정답:")), "")
        category = next((part.replace("카테고리: ", "") for part in parts if part.startswith("카테고리:")), "")
        
        new_row = {
            'id': f"augmented-{i}",
            'paragraph': paragraph,
            'question': question_text,
            'choices': choices,
            'answer': answer,
            'category': category,
            'question_plus': np.nan
        }
        augmented_df = pd.concat([augmented_df, pd.DataFrame([new_row])], ignore_index=True)
    except Exception as e:
        print(f"Error processing question {i}: {str(e)}")

augmented_df

,id,paragraph,question,choices,answer,category,question_plus
0,augmented-0,조선 후기에 들어서면서 사대부 계층의 정치적 영향력이 점차 늘어나게 되었다. 이들은...,조선 후기에 남인과 북인의 갈등이 심화된 원인으로 가장 적절한 것은 무엇인가?,"['사대부의 정치적 영향력이 커졌다', '경제적 불만이 사회 불안을 초래했다', '...",3,사회-한국사,NaN
1,augmented-1,고려시대에는 불교가 국가의 주된 사상으로 자리 잡으면서 다양한 불교 사찰과 신앙이 ...,고려시대의 불교 사상이 미친 영향에 대한 설명으로 옳은 것은 무엇인가?,"['불교의 영향으로 왕실과 국민의 통합이 이루어졌다.', '대중이 불교를 따르지 않...",1,사회-한국사,NaN
2,augmented-2,"조선 시대, 성리학이 국가의 이념으로 자리 잡으면서 교육제도의 뼈대를 형성하게 되었...",위 지문에서 설명된 조선 시대의 교육 이념과 사회에 미친 영향에 대해 가장 옳은 설...,,4,사회-한국사,NaN
3,augmented-3,고려시대에 가장 중요한 정치적 단체인 중앙관청은 국왕과 고위 관리들로 구성된 의사결...,고려시대 중앙관청의 기능과 관련하여 옳지 않은 것은 무엇인가?,"['문하시중은 군사적 권한을 보유하였다.', '왕권 강화를 위해 제도가 시행되었다....",4,사회-한국사,NaN
4,augmented-4,"조선시대에 집강소가 처음 설치된 것은 임진왜란 이후의 일로, 이를 통해 지방의 자치...",조선시대 집강소의 역할에 대한 설명으로 옳은 것은?,"['집강소는 중앙 정부의 직접 지휘를 받았다.', '지역 주민의 의견을 수렴하는 역...",2,사회-한국사,NaN
...,...,...,...,...,...,...,...
1957,augmented-1957,최근에는 젊은 세대가 다양한 경제적 문제를 겪고 있습니다. 특히 주택 가격의 급등과...,최근 젊은 세대가 겪고 있는 문제 중에서 주택 문제와 관련하여 정부가 추진하는 정책...,"['청년 실업과 채용 확대 문제', '소득 불균형 해소에 대한 정책', '세계 경제...",1,국어-사회,NaN
1958,augmented-1958,최근 한 온라인 플랫폼에서 청소년들이 '편지'라는 형식을 모른다는 이야기가 화제가 ...,청소년들이 감정을 담아 편지를 쓰지 않게 된 주된 원인은 무엇인가?,"['소통 방식의 변화와 기술 발전', '편지 쓰기의 전통적인 매력', '감사 표현의...",1,국어-작문,NaN
1959,augmented-1959,최근 동화 출판 분야에 신선한 바람을 일으킬 작품들이 속속 등장하고 있다. ‘202...,'2023 동화 창작 공모전'의 대표 수상작 제목은 무엇인가?,"['구름 위의 여행', '판타지의 모험과 심화', '어린이의 상상력과 꿈', '어린...",1,국어-문학,NaN
1960,augmented-1960,최근 들어 다양한 사회적 이슈를 담은 다큐멘터리 제작이 증가하고 있다. 다큐멘터리는...,다큐멘터리 제작에 있어 중요한 요소는 무엇인가?,"['사건의 단순 나열이 필요하다', '규칙적인 방송 시간이 중요하다', '사람의 감...",3,국어-독서,NaN


In [12]:
# 비어있는 데이터 찾기
empty_data = augmented_df[
    (augmented_df['paragraph'].isna() | (augmented_df['paragraph'] == '')) |
    (augmented_df['question'].isna() | (augmented_df['question'] == '')) |
    (augmented_df['choices'].isna() | (augmented_df['choices'] == ''))
]

# 비어있지 않은 데이터 찾기
non_empty_data = augmented_df[
    (augmented_df['paragraph'].notna() & (augmented_df['paragraph'] != '')) &
    (augmented_df['question'].notna() & (augmented_df['question'] != '')) &
    (augmented_df['choices'].notna() & (augmented_df['choices'] != ''))
]

# 비어있는 데이터와 비어있지 않은 데이터의 개수 출력
print(f"비어있는 데이터의 총 개수: {len(empty_data)}")
print(f"비어있지 않은 데이터의 총 개수: {len(non_empty_data)}")

# 각 컬럼별로 비어있는 데이터 개수 출력
for column in ['paragraph', 'question', 'choices']:
    empty_count = augmented_df[augmented_df[column].isna() | (augmented_df[column] == '')].shape[0]
    print(f"{column} 컬럼의 비어있는 데이터 개수: {empty_count}")

# 비어있는 데이터 확인 (처음 5개 행)
print("\n비어있는 데이터 샘플:")
print(empty_data.head())

# 비어있지 않은 데이터 확인 (처음 5개 행)
print("\n비어있지 않은 데이터 샘플:")
print(non_empty_data.head())

비어있는 데이터의 총 개수: 202
비어있지 않은 데이터의 총 개수: 1760
paragraph 컬럼의 비어있는 데이터 개수: 5
question 컬럼의 비어있는 데이터 개수: 7
choices 컬럼의 비어있는 데이터 개수: 201

비어있는 데이터 샘플:
              id                                          paragraph  \
2    augmented-2  조선 시대, 성리학이 국가의 이념으로 자리 잡으면서 교육제도의 뼈대를 형성하게 되었...   
14  augmented-14  6.25 전쟁 이후 대한민국은 경제 발전을 위해 여러 가지 정책을 시행해왔다. 19...   
18  augmented-18  조선 후기에는 여러 사회적 변화가 이루어졌고, 이로 인해 새로운 사회 계층이 형성되...   
26  augmented-26  조선시대 중엽, 국가의 사회적 안전망으로 일본과의 무역이 확대되었다. 중국과의 무역...   
37  augmented-37  대한민국은 1948년에 제정된 헌법을 통해 민주공화국으로서의 모습을 갖추게 되었고,...   

                                             question choices answer  \
2   위 지문에서 설명된 조선 시대의 교육 이념과 사회에 미친 영향에 대해 가장 옳은 설...            4     
14            1960년대 중반 한국 경제 발전에 대한 설명으로 옳은 것은 무엇인가?            1     
18                  상인 계층의 등장에 대한 설명 중 옳지 않은 것은 무엇인가?            1     
26       조선시대 중엽 일본과의 교류 확대가 유발한 변화로 적절하지 않은 것은 무엇인가?            4     
37               대한민국 헌법의 핵심적 기능에 대한 설명으로 옳은 것은 무엇인가?            2   

In [13]:
len(empty_data)

202

In [14]:
non_empty_data

,id,paragraph,question,choices,answer,category,question_plus
0,augmented-0,조선 후기에 들어서면서 사대부 계층의 정치적 영향력이 점차 늘어나게 되었다. 이들은...,조선 후기에 남인과 북인의 갈등이 심화된 원인으로 가장 적절한 것은 무엇인가?,"['사대부의 정치적 영향력이 커졌다', '경제적 불만이 사회 불안을 초래했다', '...",3,사회-한국사,NaN
1,augmented-1,고려시대에는 불교가 국가의 주된 사상으로 자리 잡으면서 다양한 불교 사찰과 신앙이 ...,고려시대의 불교 사상이 미친 영향에 대한 설명으로 옳은 것은 무엇인가?,"['불교의 영향으로 왕실과 국민의 통합이 이루어졌다.', '대중이 불교를 따르지 않...",1,사회-한국사,NaN
3,augmented-3,고려시대에 가장 중요한 정치적 단체인 중앙관청은 국왕과 고위 관리들로 구성된 의사결...,고려시대 중앙관청의 기능과 관련하여 옳지 않은 것은 무엇인가?,"['문하시중은 군사적 권한을 보유하였다.', '왕권 강화를 위해 제도가 시행되었다....",4,사회-한국사,NaN
4,augmented-4,"조선시대에 집강소가 처음 설치된 것은 임진왜란 이후의 일로, 이를 통해 지방의 자치...",조선시대 집강소의 역할에 대한 설명으로 옳은 것은?,"['집강소는 중앙 정부의 직접 지휘를 받았다.', '지역 주민의 의견을 수렴하는 역...",2,사회-한국사,NaN
5,augmented-5,"고려 시대는 역동적인 정치와 사회 구조의 변화를 겪으면서 다양한 문화적, 경제적 발...",이 지문에서 설명하는 고려 시대의 정치적 변화와 관련된 주요 인물로 적절한 것은 누...,"['경종과 성종의 개혁적 리더십', '후삼국 통일을 이끈 왕', '유교 이념 확립에...",1,사회-한국사,NaN
...,...,...,...,...,...,...,...
1957,augmented-1957,최근에는 젊은 세대가 다양한 경제적 문제를 겪고 있습니다. 특히 주택 가격의 급등과...,최근 젊은 세대가 겪고 있는 문제 중에서 주택 문제와 관련하여 정부가 추진하는 정책...,"['청년 실업과 채용 확대 문제', '소득 불균형 해소에 대한 정책', '세계 경제...",1,국어-사회,NaN
1958,augmented-1958,최근 한 온라인 플랫폼에서 청소년들이 '편지'라는 형식을 모른다는 이야기가 화제가 ...,청소년들이 감정을 담아 편지를 쓰지 않게 된 주된 원인은 무엇인가?,"['소통 방식의 변화와 기술 발전', '편지 쓰기의 전통적인 매력', '감사 표현의...",1,국어-작문,NaN
1959,augmented-1959,최근 동화 출판 분야에 신선한 바람을 일으킬 작품들이 속속 등장하고 있다. ‘202...,'2023 동화 창작 공모전'의 대표 수상작 제목은 무엇인가?,"['구름 위의 여행', '판타지의 모험과 심화', '어린이의 상상력과 꿈', '어린...",1,국어-문학,NaN
1960,augmented-1960,최근 들어 다양한 사회적 이슈를 담은 다큐멘터리 제작이 증가하고 있다. 다큐멘터리는...,다큐멘터리 제작에 있어 중요한 요소는 무엇인가?,"['사건의 단순 나열이 필요하다', '규칙적인 방송 시간이 중요하다', '사람의 감...",3,국어-독서,NaN


In [15]:
# answer 컬럼의 고유한 값 확인
unique_answers = non_empty_data['answer'].unique()
print("answer 컬럼의 고유한 값:", unique_answers.tolist())

# answer 컬럼의 값 분포 확인
answer_distribution = non_empty_data['answer'].value_counts()
print("\nanswer 컬럼의 값 분포:")
print(answer_distribution)

answer 컬럼의 고유한 값: ['3', '1', '4', '2', '2  ', '1  ', '4  ', '5', '5 ', '1 ', '3  ', '5  ', '2 ', '3 ', '4 ']

answer 컬럼의 값 분포:
answer
1      811
2      339
3      281
4       98
1       87
5       46
2       40
3       28
4       10
5        8
1        6
5        2
2        2
3        1
4        1
Name: count, dtype: int64


In [16]:
import pandas as pd

# 공백 제거 함수 정의
def strip_whitespace(value):
    return value.strip() if isinstance(value, str) else value

# 'answer' 컬럼의 모든 값에 대해 공백 제거 적용
non_empty_data['answer'] = non_empty_data['answer'].apply(strip_whitespace)

/tmp/ipykernel_247074/1230353517.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_empty_data['answer'] = non_empty_data['answer'].apply(strip_whitespace)


In [17]:
# answer 컬럼의 고유한 값 확인
unique_answers = non_empty_data['answer'].unique()
print("answer 컬럼의 고유한 값:", unique_answers.tolist())

# answer 컬럼의 값 분포 확인
answer_distribution = non_empty_data['answer'].value_counts()
print("\nanswer 컬럼의 값 분포:")
print(answer_distribution)

answer 컬럼의 고유한 값: ['3', '1', '4', '2', '5']

answer 컬럼의 값 분포:
answer
1    904
2    381
3    310
4    109
5     56
Name: count, dtype: int64


In [18]:
non_empty_data

,id,paragraph,question,choices,answer,category,question_plus
0,augmented-0,조선 후기에 들어서면서 사대부 계층의 정치적 영향력이 점차 늘어나게 되었다. 이들은...,조선 후기에 남인과 북인의 갈등이 심화된 원인으로 가장 적절한 것은 무엇인가?,"['사대부의 정치적 영향력이 커졌다', '경제적 불만이 사회 불안을 초래했다', '...",3,사회-한국사,NaN
1,augmented-1,고려시대에는 불교가 국가의 주된 사상으로 자리 잡으면서 다양한 불교 사찰과 신앙이 ...,고려시대의 불교 사상이 미친 영향에 대한 설명으로 옳은 것은 무엇인가?,"['불교의 영향으로 왕실과 국민의 통합이 이루어졌다.', '대중이 불교를 따르지 않...",1,사회-한국사,NaN
3,augmented-3,고려시대에 가장 중요한 정치적 단체인 중앙관청은 국왕과 고위 관리들로 구성된 의사결...,고려시대 중앙관청의 기능과 관련하여 옳지 않은 것은 무엇인가?,"['문하시중은 군사적 권한을 보유하였다.', '왕권 강화를 위해 제도가 시행되었다....",4,사회-한국사,NaN
4,augmented-4,"조선시대에 집강소가 처음 설치된 것은 임진왜란 이후의 일로, 이를 통해 지방의 자치...",조선시대 집강소의 역할에 대한 설명으로 옳은 것은?,"['집강소는 중앙 정부의 직접 지휘를 받았다.', '지역 주민의 의견을 수렴하는 역...",2,사회-한국사,NaN
5,augmented-5,"고려 시대는 역동적인 정치와 사회 구조의 변화를 겪으면서 다양한 문화적, 경제적 발...",이 지문에서 설명하는 고려 시대의 정치적 변화와 관련된 주요 인물로 적절한 것은 누...,"['경종과 성종의 개혁적 리더십', '후삼국 통일을 이끈 왕', '유교 이념 확립에...",1,사회-한국사,NaN
...,...,...,...,...,...,...,...
1957,augmented-1957,최근에는 젊은 세대가 다양한 경제적 문제를 겪고 있습니다. 특히 주택 가격의 급등과...,최근 젊은 세대가 겪고 있는 문제 중에서 주택 문제와 관련하여 정부가 추진하는 정책...,"['청년 실업과 채용 확대 문제', '소득 불균형 해소에 대한 정책', '세계 경제...",1,국어-사회,NaN
1958,augmented-1958,최근 한 온라인 플랫폼에서 청소년들이 '편지'라는 형식을 모른다는 이야기가 화제가 ...,청소년들이 감정을 담아 편지를 쓰지 않게 된 주된 원인은 무엇인가?,"['소통 방식의 변화와 기술 발전', '편지 쓰기의 전통적인 매력', '감사 표현의...",1,국어-작문,NaN
1959,augmented-1959,최근 동화 출판 분야에 신선한 바람을 일으킬 작품들이 속속 등장하고 있다. ‘202...,'2023 동화 창작 공모전'의 대표 수상작 제목은 무엇인가?,"['구름 위의 여행', '판타지의 모험과 심화', '어린이의 상상력과 꿈', '어린...",1,국어-문학,NaN
1960,augmented-1960,최근 들어 다양한 사회적 이슈를 담은 다큐멘터리 제작이 증가하고 있다. 다큐멘터리는...,다큐멘터리 제작에 있어 중요한 요소는 무엇인가?,"['사건의 단순 나열이 필요하다', '규칙적인 방송 시간이 중요하다', '사람의 감...",3,국어-독서,NaN


In [19]:
# '사회'가 포함된 행의 비율 계산
social_ratio = non_empty_data['category'].str.contains('사회').mean()

# '국어'가 포함된 행의 비율 계산
korean_ratio = non_empty_data['category'].str.contains('국어').mean()

print(f"'사회' 비율: {social_ratio:.2%}")
print(f"'국어' 비율: {korean_ratio:.2%}")

# '사회'와 '국어'의 비율 계산
if korean_ratio > 0:
    overall_ratio = social_ratio / korean_ratio
    print(f"'사회'/'국어' 비율: {overall_ratio:.2f}")
else:
    print("'국어' 카테고리가 없어 비율을 계산할 수 없습니다.")

'사회' 비율: 92.78%
'국어' 비율: 5.85%
'사회'/'국어' 비율: 15.85


In [20]:
non_empty_data.to_csv("test_1000data.csv", index=False)